### Goal of this notebook is to process raw business survival rate data such that it can be appended to the Population nodes in the Knowledge graph

In [16]:
import pandas as pd

df = pd.read_csv("../data/raw/business-survival-rates.csv", na_values=[';', 'NA', ':', ''])

print(df.shape)
print("\nNull values per column:")
df.isna().sum()

(1020, 14)

Null values per column:


code                        0
area                        0
year                        0
births                      0
1_year_survival_number      0
1_year_survival_rate        0
2_year_survival_number     51
2_year_survival_rate       51
3_year_survival_number    102
3_year_survival_rate      102
4_year_survival_number    153
4_year_survival_rate      153
5_year_survival_number    204
5_year_survival_rate      204
dtype: int64

In [17]:
years = df['year'].dropna().unique()
years.sort()
boroughs = df['area'].dropna().unique()
boroughs.sort()

print(f"\nYear range: {years[0]} - {years[-1]}")
print(f"Number of boroughs: {len(boroughs)}")


Year range: 2002 - 2021
Number of boroughs: 51


In [22]:
numeric_cols = df.select_dtypes(include='number').columns
for col in numeric_cols:
    if (df[col] < 0).any():
        print(f"Warning: Negative values found in column '{col}'")

# 6. Show summary statistics
print("\nSummary statistics:")
df.drop(columns=['code', 'area', 'year']).describe()


Summary statistics:


,births,1_year_survival_number,1_year_survival_rate,2_year_survival_number,2_year_survival_rate,3_year_survival_number,3_year_survival_rate,4_year_survival_number,4_year_survival_rate,5_year_survival_number,5_year_survival_rate
count,1020.000000,1020.000000,1020.000000,969.000000,969.000000,918.000000,918.000000,867.000000,867.000000,816.000000,816.000000
mean,31464.833333,29271.906863,92.721863,23345.784314,74.449536,18388.382353,58.541612,15109.607843,48.366897,12772.708333,41.065319
std,78218.477841,72921.977582,3.492929,57964.803993,4.327230,45672.562878,4.695703,37576.221390,4.451732,31818.114641,4.174375
min,435.000000,410.000000,60.500000,335.000000,46.800000,250.000000,33.400000,205.000000,21.500000,170.000000,16.600000
25%,1575.000000,1445.000000,92.100000,1145.000000,72.300000,890.000000,55.800000,725.000000,45.800000,600.000000,38.700000
50%,2517.500000,2330.000000,93.500000,1815.000000,74.500000,1400.000000,58.600000,1130.000000,48.600000,942.500000,41.200000
75%,20406.250000,19268.750000,94.700000,15550.000000,77.200000,12386.250000,61.600000,10140.000000,51.200000,8615.000000,43.600000
max,397540.000000,378505.000000,100.000000,282775.000000,88.200000,215085.000000,71.100000,178505.000000,62.500000,152495.000000,56.200000


In [24]:
businesses_data_file_path = '../data/processed/businesses_with_boroughs.csv'
businesses_df = pd.read_csv(businesses_data_file_path)
businesses_boroughs = businesses_df['area'].dropna().unique()
print(f"Number of boroughs in businesses dataset: {len(businesses_boroughs)}")

# Compare with survival rates dataset boroughs
print(f"Number of boroughs in survival rates dataset: {len(boroughs)}")

# Boroughs in businesses but not in survival rates
missing_in_survival = set(businesses_boroughs) - set(boroughs)
print(f"Boroughs in businesses but not in survival rates: {missing_in_survival}")

# Boroughs in survival rates but not in businesses
missing_in_businesses = set(boroughs) - set(businesses_boroughs)
print(f"Boroughs in survival rates but not in businesses: {missing_in_businesses}")

Number of boroughs in businesses dataset: 33
Number of boroughs in survival rates dataset: 51
Boroughs in businesses but not in survival rates: set()
Boroughs in survival rates but not in businesses: {'Inner London', 'Scotland', 'West Midlands', 'South West', 'South East', 'North East', 'Great Britain', 'North West', 'England', 'Yorkshire And The Humber', 'East', 'London', 'East Midlands', 'Wales', 'United Kingdom', 'Outer London', 'England And Wales', 'Northern Ireland'}


TODO: remove irrelevant 'boroughs' from survival rates dataset. Perhaps do keep aggregates and nation-wide: [London, Inner/Outer London, Great Britain]